# 🛤️ Proyecto: Rutas de Patrullaje "Festival del Viento"

Este notebook analiza la viabilidad de una ruta de patrullaje óptima en un festival utilizando la **Teoría de Grafos**, específicamente el concepto de **Circuitos Eulerianos**. 

El objetivo es determinar si un guardia puede **recorrer cada sendero exactamente una vez** y regresar al inicio, y proponer una solución si no es posible.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/scysco/Essentials/blob/main/graph_theory/pj_rutas_festival/pj_rutas_festival.ipynb)

## 🛠️ 1. Instalación de Dependencias

Solo necesitamos `networkx` para el grafo y `matplotlib` para dibujarlo.

In [ ]:
!pip install networkx matplotlib

## 📦 2. Importación de Librerías

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import warnings

# Ocultar advertencias futuras de Matplotlib/NetworkX para una salida más limpia
warnings.filterwarnings("ignore", category=FutureWarning)

## 📊 3. Grafo Original: Análisis Euleriano

Primero, modelamos el festival como un **Grafo No Dirigido** (`nx.Graph`) y definimos sus zonas (nodos) y senderos (aristas).

In [ ]:
# 1. Crear el Grafo Original
G_orig = nx.Graph()

# 2. Definir Nodos (Zonas)
node_labels = {
    'EP': 'Entrada Principal',
    'ES': 'Escenario Sol',
    'ZG': 'Zona Gastronómica',
    'EL': 'Escenario Luna',
    'AD': 'Área de Descanso',
    'ML': 'Muelle'
}
G_orig.add_nodes_from(node_labels.keys())

# 3. Definir Aristas (Senderos)
edges_list_orig = [
    ('EP', 'ES'), ('EP', 'ZG'), ('ES', 'ZG'), ('ES', 'EL'), 
    ('ZG', 'AD'), ('EL', 'AD'), ('EL', 'ML'), ('AD', 'ML')
]
G_orig.add_edges_from(edges_list_orig)

# 4. Calcular Grados
degrees_orig = dict(G_orig.degree())

# 5. Crear Etiquetas y Colores basados en Grados
custom_labels_orig = {}
node_colors_orig = []
for node, name in node_labels.items():
    degree = degrees_orig[node]
    custom_labels_orig[node] = f"{name}\n(Grado: {degree})"
    # Resaltar nodos con grado impar (el problema)
    if degree % 2 != 0:
        node_colors_orig.append('#e53935') # Rojo
    else:
        node_colors_orig.append('#43a047') # Verde

### Teorema del Circuito Euleriano

Un grafo tiene un **Circuito Euleriano** (empezar, recorrer todas las aristas 1 vez, y terminar en el inicio) si y solo si:
1.  Está conectado.
2.  **Todos** sus nodos tienen **grado par**.

Vamos a comprobarlo:

In [ ]:
# Comprobar la condición Euleriana
is_eulerian = all(degree % 2 == 0 for degree in degrees_orig.values())

print(f"--- ANÁLISIS DEL GRAFO ORIGINAL ---")
print(f"Grados de cada nodo: {degrees_orig}")
print(f"¿Tiene un Circuito Euleriano? {is_eulerian}")

if not is_eulerian:
    odd_nodes = [node for node, degree in degrees_orig.items() if degree % 2 != 0]
    print(f"Problema: Los nodos {odd_nodes} tienen grado impar.")

### Visualización (Original)

Los nodos en **rojo** son los que tienen grado impar, impidiendo la ruta.

In [ ]:
# Dibujar el Grafo Original
pos_orig = nx.spring_layout(G_orig, seed=42, k=0.8)
plt.figure(figsize=(14, 9))

nx.draw(
    G_orig,
    pos=pos_orig,
    labels=custom_labels_orig,
    with_labels=True,
    node_color=node_colors_orig,
    node_size=3500,
    font_size=10,
    font_weight='bold',
    font_color='black',
    edge_color='gray',
    width=2
)

plt.title('Grafo del Festival (Original - No Euleriano)', fontsize=16)
plt.savefig("grafo_festival_original.png", bbox_inches='tight')
plt.show()

## 🔄 4. Grafo Modificado: Propuesta de Solución

Para que el grafo sea Euleriano, debemos hacer que los 4 nodos impares (`ES`, `ZG`, `EL`, `AD`) tengan grado par. La forma más simple es añadir aristas entre pares de ellos.

**Solución Propuesta:** Añadir un nuevo sendero (arista) entre la **Zona Gastronómica (`ZG`)** y el **Escenario Luna (`EL`)**. 

* Grado de `ZG` pasa de 3 a 4 (par).
* Grado de `EL` pasa de 3 a 4 (par).

*(Nota: Aún quedan `ES` y `AD` con grado impar. El script `graph_festival.py` original tenía un error, solo añadía 1 arista. Para hacerlo Euleriano, necesitamos añadir OTRA arista, por ejemplo, entre `ES` y `AD`)*.

**ACTUALIZACIÓN:** Basado en el `graph_festival.py` del repositorio, la solución propuesta añade **dos aristas**: `('ZG', 'EL')` y `('ES', 'AD')`.

In [ ]:
# 1. Copiar el grafo original
G_mod = G_orig.copy()

# 2. Añadir las nuevas aristas (senderos) para arreglar los 4 nodos impares
new_edges = [('ZG', 'EL'), ('ES', 'AD')]
G_mod.add_edges_from(new_edges)

# 3. Recalcular Grados
degrees_mod = dict(G_mod.degree())

# 4. Crear Etiquetas y Colores
custom_labels_mod = {}
node_colors_mod = []
for node, name in node_labels.items():
    degree = degrees_mod[node]
    custom_labels_mod[node] = f"{name}\n(Grado: {degree})"
    # Ahora todos deberían ser pares
    if degree % 2 != 0:
        node_colors_mod.append('#e53935') # Rojo (error)
    else:
        node_colors_mod.append('#43a047') # Verde (éxito)

# 5. Comprobar la condición Euleriana de nuevo
is_eulerian_mod = all(degree % 2 == 0 for degree in degrees_mod.values())
print(f"--- ANÁLISIS DEL GRAFO MODIFICADO ---")
print(f"Grados de cada nodo: {degrees_mod}")
print(f"¿Tiene un Circuito Euleriano? {is_eulerian_mod}")

### Visualización (Modificado)

Con los nuevos senderos (en azul), todos los nodos son **verdes** (grado par), lo que confirma que la ruta de patrullaje ideal **ahora es posible**.

In [ ]:
# Dibujar el Grafo Modificado
pos_mod = nx.spring_layout(G_mod, seed=42, k=0.8)
plt.figure(figsize=(14, 9))

# Dibujar nodos y aristas antiguas
nx.draw(
    G_mod,
    pos=pos_mod,
    labels=custom_labels_mod,
    with_labels=True,
    node_color=node_colors_mod,
    node_size=3500,
    font_size=10,
    font_weight='bold',
    font_color='black',
    edge_color='gray',
    width=2
)

# Resaltar las nuevas aristas añadidas
nx.draw_networkx_edges(
    G_mod,
    pos=pos_mod,
    edgelist=new_edges,
    edge_color='#0277bd', # Azul
    width=3,
    style='dashed'
)

plt.title('Grafo del Festival (Modificado - ¡Euleriano!)', fontsize=16)
plt.savefig("grafo_festival_modificado.png", bbox_inches='tight')
plt.show()